<a href="https://colab.research.google.com/github/Bormey-Sky/Text_Summarization/blob/main/Updated_Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# wrap output text into colab cell
from IPython.display import HTML, display


def set_css():
  display(HTML('''
    <style>
      pre {
        white-space: pre-wrap;
      }
    </style>
  '''))

get_ipython().events.register('pre_run_cell', set_css)


In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def delete_files():
    # Get all files in the folder
    all_files = os.listdir(folder_path)

    # If no files are uploaded
    if not all_files:
        print("No files are uploaded.")
    else:
        print("Uploaded files:")
        for i, file_name in enumerate(all_files):
            print(f"{i+1}. {file_name}")

        choice = input("Do you want to delete all files? (yes/no): ").lower()

        if choice == "yes":
            for file_name in all_files:
                os.remove(os.path.join(folder_path, file_name))
            print("All files deleted.")
        else:
            selected_files_indices = input("Type the numbers of the files you want to delete (separated by comma): ").split(',')

            for file_index in selected_files_indices:
                file_index = int(file_index.strip()) - 1  # Subtract 1 because the list is zero-indexed
                if file_index < len(all_files):
                    file_name = all_files[file_index]
                    os.remove(os.path.join(folder_path, file_name))
                    print(f"Deleted {file_name}.")
                else:
                    print(f"No file corresponding to the index {file_index+1}.")


In [ ]:
from google.colab import files
import shutil
import os

# from google.colab import drive
# drive.mount('/content/drive')

# Path to the drive folder
folder_path = "/content/drive/MyDrive/docs_to_summarize"

# If the folder does not exist, create it
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

while True:
    print("Upload your files (.docx, .txt). When finished, press Cancel and then Enter.")
    uploaded_files = files.upload()

    for file_name in uploaded_files.keys():
        # Only move the files with the correct extensions
        if file_name.endswith(".docx") or file_name.endswith(".txt") or file_name.endswith(".pdf"):
            # Get the path of the uploaded file
            src_path = os.path.join("/content", file_name)
            # Get the destination path
            dest_path = os.path.join(folder_path, file_name)
            # Move the file to the drive folder
            shutil.move(src_path, dest_path)

    # Ask the user whether to continue uploading
    should_continue = input("Do you want to upload more files? (yes/no): ").lower()
    if should_continue != "yes":
        break


Upload your files (.docx, .txt). When finished, press Cancel and then Enter.


Saving Can Cambodia join Vietnam as a reliable ally of South Korea_.docx to Can Cambodia join Vietnam as a reliable ally of South Korea_.docx
Do you want to upload more files? (yes/no): yes
Upload your files (.docx, .txt). When finished, press Cancel and then Enter.


Saving Four Suspects Arrested Following Hotel Fire Which Killed Eight.txt to Four Suspects Arrested Following Hotel Fire Which Killed Eight.txt
Do you want to upload more files? (yes/no): yes
Upload your files (.docx, .txt). When finished, press Cancel and then Enter.


Saving Candlelight Leader Rong Chhun Postpones Mass Protest After Hun Sen Threatens To Arrest Him.txt to Candlelight Leader Rong Chhun Postpones Mass Protest After Hun Sen Threatens To Arrest Him.txt
Do you want to upload more files? (yes/no): no


In [ ]:
# delete_files() # if u want to delete any or all files

Uploaded files:
1. Will AI really destroy humanity_.txt
2. .ipynb_checkpoints
3. camboda_ban_meta_board.docx
4. Representatives from the ruling Cambodian People’s Party (CPP) and the Grassroots Democratic Party (GDP) gave the audience a summary of their policies as they prepare for the National Election 2023, touching on issues like the economy an.docx
5. Can Cambodia join Vietnam as a reliable ally of South Korea_ (1).docx
6. Four Suspects Arrested Following Hotel Fire Which Killed Eight.txt
Do you want to delete all files? (yes/no): 1,3,4,5,6
Type the numbers of the files you want to delete (separated by comma): 1,3,4,5,6
Deleted Will AI really destroy humanity_.txt.
Deleted camboda_ban_meta_board.docx.
Deleted Representatives from the ruling Cambodian People’s Party (CPP) and the Grassroots Democratic Party (GDP) gave the audience a summary of their policies as they prepare for the National Election 2023, touching on issues like the economy an.docx.
Deleted Can Cambodia join Vietnam

In [ ]:
!pip install python-docx pyPDF2 transformers nltk bert-extractive-summarizer --quiet
import nltk
nltk.download('punkt')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 52.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 25.0 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from docx import Document

def read_docx(file_path):
    doc = Document(file_path)
    return " ".join([paragraph.text for paragraph in doc.paragraphs])

In [ ]:
# import PyPDF2

# def read_pdf(file_path):
#     from PyPDF2 import PdfReader
#     with open(file_path, "rb") as file:
#         pdf = PdfReader(file)
#         text = ""
#         for page in pdf.pages:
#             text += page.extract_text()
#     return text

In [ ]:
def read_txt(file_path):
    with open(file_path, 'r') as file:
        return file.read().replace('\n', '')

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
# Extractive summarization using KMEAN to rank sentences and below model to extract text embeddings
# Reference: https://pypi.org/project/bert-extractive-summarizer/#custom-model-example
# CNN for extract embeddings: https://huggingface.co/facebook/bart-large-cnn

# Init extractive model
from transformers import AutoConfig, AutoModel, AutoTokenizer, pipeline
from summarizer import Summarizer
from nltk import sent_tokenize

# Load model, model config and tokenizer via Transformers
model_path ='drive/MyDrive/fb-bart-large-cnn'

custom_config = AutoConfig.from_pretrained(model_path)
custom_config.output_hidden_states=True
custom_tokenizer = AutoTokenizer.from_pretrained(model_path)
custom_model = AutoModel.from_pretrained(model_path, config=custom_config)

model = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)

In [ ]:
# Init abstract model
from transformers import pipeline

summarizer = pipeline("summarization", model='drive/MyDrive/fb-bart-large-cnn')

In [ ]:
def summary_files():
    # Get all files in the folder
    all_files = os.listdir(folder_path)

    # Filter files with valid extensions
    valid_files = [file for file in all_files if file.endswith(('.docx', '.txt'))]

    # If no files are uploaded
    if not valid_files:
        print("No valid files are uploaded.")
    else:
        print("Uploaded files:")
        for i, file_name in enumerate(valid_files):
            print(f"{i+1}. {file_name}")

        choice = input("\nDo you want to summarize all files? (yes/no): ").lower()

        if choice == "yes":
            selected_files_indices = range(len(valid_files))
        else:
            selected_files_indices = [int(x.strip()) - 1 for x in input("Type the numbers of the files you want to process (separated by comma): ").split(',')]

        # Ask user for the number of sentences for the extractive summary
        num_sentences = int(input("\nEnter the number of sentences for the extractive summary: "))

        for file_index in selected_files_indices:
            if file_index < len(valid_files):
                file_name = valid_files[file_index]
                print(f"\nSummarizing {file_name}...")
                file_path = os.path.join(folder_path, file_name)

                # Determine the file type and read the content accordingly
                if file_name.endswith(".docx"):
                    input_text = read_docx(file_path)
                elif file_name.endswith(".txt"):
                    input_text = read_txt(file_path)

                # Calculate the number of tokens in the input text
                tokens = custom_tokenizer.encode(input_text)
                max_length = len(tokens)

                # Perform summarization on the content of each file
                # Extractive summarization
                result = model(input_text, num_sentences=num_sentences, max_length=max_length)

                # Print the extractive summary
                sentences = sent_tokenize(result)
                print("\nExtractive Summary:")
                for i, sentence in enumerate(sentences):
                    print(f"{sentence}")

                # Abstractive summarization
                result = summarizer(input_text, max_length=max_length, min_length=30, do_sample=False)[0]['summary_text']

                # Print the abstractive summary
                sentences = sent_tokenize(result)
                print("\nAbstractive Summary:")
                for i, sentence in enumerate(sentences):
                    print(f"{sentence}")
            else:
                print(f"No file corresponding to the index {file_index+1}.")

summary_files()


Uploaded files:
1. Can Cambodia join Vietnam as a reliable ally of South Korea_ (1).docx
2. Can Cambodia join Vietnam as a reliable ally of South Korea_.docx
3. Four Suspects Arrested Following Hotel Fire Which Killed Eight.txt
4. Candlelight Leader Rong Chhun Postpones Mass Protest After Hun Sen Threatens To Arrest Him.txt

Do you want to summarize all files? (yes/no): 2,3,4
Type the numbers of the files you want to process (separated by comma): 2,3,4

Enter the number of sentences for the extractive summary: 4

Summarizing Can Cambodia join Vietnam as a reliable ally of South Korea_.docx...

Extractive Summary:
Can Cambodia join Vietnam as a reliable ally of South Korea?
Vietnam is seen as having a highly wise foreign policy because it is able to maintain cordial ties with all nations, including the US, Russia, Japan, the EU, Australia, and South Korea.
Promote success stories: Highlighting successful South Korean investments in Vietnam and their positive outcomes can showcase the co

IndexError: ignored

In [ ]:
# # Extractive summary
# from transformers import pipeline

# summarizer = pipeline("summarization", model='drive/MyDrive/fb-bart-large-cnn')

# input_text = """
# The Cambodia Government has banned 22 members of the Oversight Board to Meta Platforms from entering the Kingdom, and if they are in the country they are required to leave immediately, within 48 hours, as of four o’clock on Tuesday.

# The statement issued yesterday by the Ministry of Foreign Affairs and International Cooperation stated that the recommendation by the Oversight Board to Meta Platforms, to temporarily suspend the official Facebook page belonging to Prime Minister Hun Sen, is political in nature.

# It intends to obstruct the freedom of the press for the citizens of Cambodia and the right to receive credible news from a leader whom they support and admire, the statement said.

# In the spirit of unwavering adherence to upholding the principle of sovereignty, and to prevent the interference into the internal affairs of Cambodia, the Government has decided to declare the following members of the Oversight Board to Meta Platforms, as ‘Persona Non Grata’, it said.

# The 22 members are: Kenji Yoshino, Julie Owono, Pamela San Martin, Ronaldo Lemos, Michael McConnell, Nighat Dad, Katherine Chen, Andras Sajo, Emi Palmor, John Samples, Khaled Mansour, Suzanne Nossel, Tawakkol Karman, Catalina Botero-Marino, Afia Asantewaa Asare-Kyei, Nicolas Suzor, Paolo Carozza, Evelyn Aswad, Sudhir Krishnaswamy, Helle Thorning-Schmidt, Alan Rusbridger and Endy Bayuni, it said.

# “The above-mentioned persons are barred from entering or remaining in the Kingdom. They are required to leave Cambodian territory within 48 hours from the issuing time of this statement, precisely on 04 July 2023 at 16:00,” the Ministry added.

# The statement came after the board linked to the social media company Meta had recommended a six-month suspension of Prime Minister Hun Sen’s Facebook and Instagram accounts for an incendiary speech he posted earlier this year threatening to use “a stick” against opposition opponents.

# The decision was likely pre-empted by the Prime Minister who announced late Wednesday night that he was going to primarily use Telegram to communicate with Cambodians and stop using Facebook.

# Meta’s Oversight Board ruled on June 29 that a January 8 speech which Mr Hun Sen gave in Kampong Cham province violated Facebook’s community standards by threatening violence against political opponents.

# “There are two choices, the first is to use legal action, the second is to use a stick. This means that I will gather all the CPP’s supporters to hold a demonstration and hit you. So, which one do you prefer?” Mr Hun Sen had said in January.

# The Oversight Board rejected Meta’s decision to keep the video on the platform for its newsworthiness, saying Mr Hun Sen’s reach on social media allowed the threat to spread more widely.

# The Ministry found irregularities related to Facebook, including the existence of fake accounts, the risk of losing private data, the use and collection of personal information, the spread of fake news, the lack of accountability and transparency, and interferences into the country’s internal politics, it said in a statement.

# In a Telegram voice message released on June 30, the Premier said he is “Not stupid enough to ban one of the most used social media platforms in Cambodia”.

# “I only deleted my own account,” Mr Hun Sen said.

# He reaffirmed that it was his own reasons, including the existence of fake accounts with his name that drove him to make the decision and not the intention by the Meta Oversight Board to consider a six-month suspension of his account.
# """

# # Perform summarization on the content of each file
# # Extractive summarization
# result = model(input_text, num_sentences=3)

# # Print the extractive summary
# sentences = sent_tokenize(result)
# print("Extractive Summary:")
# for i, sentence in enumerate(sentences):
#     print(f"{sentence}")

Extractive Summary:
The Cambodia Government has banned 22 members of the Oversight Board to Meta Platforms from entering the Kingdom, and if they are in the country they are required to leave immediately, within 48 hours, as of four o’clock on Tuesday.
The 22 members are: Kenji Yoshino, Julie Owono, Pamela San Martin, Ronaldo Lemos, Michael McConnell, Nighat Dad, Katherine Chen, Andras Sajo, Emi Palmor, John Samples, Khaled Mansour, Suzanne Nossel, Tawakkol Karman, Catalina Botero-Marino, Afia Asantewaa Asare-Kyei, Nicolas Suzor, Paolo Carozza, Evelyn Aswad, Sudhir Krishnaswamy, Helle Thorning-Schmidt, Alan Rusbridger and Endy Bayuni, it said.
In a Telegram voice message released on June 30, the Premier said he is “Not stupid enough to ban one of the most used social media platforms in Cambodia”.


In [ ]:
# Define the input text
input_text = """

Amid the uncertainty on social media after he ordered the deletion of his official Facebook page, Prime Minister Hun Sen made it clear that he will not ban the platform in Cambodia.

In a Telegram voice message released on Friday, the Premier said he is “not stupid enough” to ban one of the most used social media platforms in Cambodia.

“I only deleted my own account,” Mr Hun Sen said.

He reaffirmed that it was his own reasons, including the existence of fake accounts with his name that drove him to make the decision and not the intention by the Meta Oversight Board to consider a six-month suspension of his account.

However, Mr Hun Sen said he is mulling taking some actions in response, including not allowing Meta to have its representatives in Cambodia.

“Facebook (Meta) has also financially benefited from Cambodian users,” Mr Hun Sen noted.

“The Ministry hereby orders Facebook representatives to leave Cambodia and cease all forms of their activities, including the company’s representation, contact with the Royal government and partnership in private sector and so on,” the statement added.

An oversight board linked to the social media company Meta had recommended a six-month suspension of Prime Minister Hun Sen’s Facebook and Instagram accounts for an incendiary speech he posted earlier this year threatening to use “a stick” against opposition opponents.

The decision was likely preempted by the Prime Minister who announced late Wednesday night that he was going to primarily use Telegram to communicate with Cambodians and stop posting on Facebook.

Meta’s Oversight Board ruled on June 29 that a January 8 speech which Mr Hun Sen gave in Kampong Cham province violated Facebook’s community standards by threatening violence against political opponents.

“There are two choices: the first is to use legal action,” Mr Hun Sen had said in January. “And the second is to use a stick. This means that I will gather all the CPP’s supporters to hold a demonstration and hit you. So, which one do you prefer?”

The Oversight Board rejected Meta’s decision to keep the video on the platform for its newsworthiness, saying Mr Hun Sen’s reach on social media allowed the threat to spread more widely.

“The Board is also concerned that a political leader’s sustained campaign of harassment and intimidation against independent media and the political opposition can become a factor in a newsworthiness assessment that leads to violating content not being removed and the account avoiding penalties,” the ruling reads.

Yang Peou, Secretary-General of the Royal Academy of Cambodia, said yesterday that Cambodians are using Facebook just as much as they eat daily, and they even benefit from information about government policies.

Personal benefits to citizens such as personal product promotion, online classes and videos about modern methods regarding agriculture, trade and various services, he said.

“If this system is shut down, how can Cambodians connect with relatives, view forums and express themselves?” Peou said. “As for public servants, they also get a lot of benefits, as the platform helps to inform about their shortcomings through the reactions from the people.”

“It is also a medium to spread policies, measures, state laws, information about cultural heritage and tourism. It can also be used to keeping abreast of social trends or behaviour,” he added.

There were 12,969,900 Facebook users in Cambodia as of January last year, which accounted for 73.9% of its entire population.

Ministry of Posts and Telecommunications undersecretary of state and spokesman Meas Po could not be reached for comment yesterday.

In a letter issued on Thursday, the Ministry of Posts and Telecommunications said it had ordered the representatives of Facebook to leave Cambodia immediately.

The announcement came after Mr Hun Sen in a Telegram voice message mentioned his response against Meta, the parent company of Facebook.

“The Ministry found irregularities related to Facebook, including the existence of fake accounts, the risk of losing private data, the use and collection of personal information, the spread of fake news, the lack of accountability and transparency, and interferences into the country’s internal politics,” it said in a statement.


"""

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers nltk bert-extractive-summarizer --quiet
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
# Extractive summarization using KMEAN to rank sentences and below model to extract text embeddings
# Reference: https://pypi.org/project/bert-extractive-summarizer/#custom-model-example
# CNN for extract embeddings: https://huggingface.co/facebook/bart-large-cnn
from transformers import AutoConfig, AutoModel, AutoTokenizer

# Load model, model config and tokenizer via Transformers
model_path ='drive/MyDrive/fb-bart-large-cnn'

custom_config = AutoConfig.from_pretrained(model_path)
custom_config.output_hidden_states=True
custom_tokenizer = AutoTokenizer.from_pretrained(model_path)
custom_model = AutoModel.from_pretrained(model_path, config=custom_config)

from summarizer import Summarizer

model = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)
result = model(input_text, num_sentences=10)

# Print the extractive summary
# Split the result into sentences
from nltk import sent_tokenize

sentences = sent_tokenize(result)
print("Extractive Summary:")

# Print each sentence
for i, sentence in enumerate(sentences):
    print(f"{sentence}")

Extractive Summary:
Amid the uncertainty on social media after he ordered the deletion of his official Facebook page, Prime Minister Hun Sen made it clear that he will not ban the platform in Cambodia.
“I only deleted my own account,” Mr Hun Sen said.
An oversight board linked to the social media company Meta had recommended a six-month suspension of Prime Minister Hun Sen’s Facebook and Instagram accounts for an incendiary speech he posted earlier this year threatening to use “a stick” against opposition opponents.
The decision was likely preempted by the Prime Minister who announced late Wednesday night that he was going to primarily use Telegram to communicate with Cambodians and stop posting on Facebook.
This means that I will gather all the CPP’s supporters to hold a demonstration and hit you.
“The Board is also concerned that a political leader’s sustained campaign of harassment and intimidation against independent media and the political opposition can become a factor in a newsw

In [ ]:
from transformers import pipeline
summarizer = pipeline("summarization", model="slauw87/bart_summarisation")

result = summarizer(input_text, max_length=1024, min_length=30, do_sample=False)[0]['summary_text']

# Print the abstractive summary
# Split the result into sentences
from nltk import sent_tokenize

sentences = sent_tokenize(result)
print("Abstractive Summary:")

# Print each sentence
for i, sentence in enumerate(sentences):
    print(f"{sentence}")

Your max_length is set to 1024, but your input_length is only 905. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=452)


Abstractive Summary:
Prime Minister Hun Sen deleted his official Facebook page.
Meta Oversight Board recommended a six-month suspension of his Facebook and Instagram accounts for an incendiary speech he posted earlier this year threatening to use "a stick" against opposition opponents.
The Ministry of Posts and Telecommunications ordered Facebook representatives to leave Cambodia immediately.
